# Imports

In [1]:
# %matplotlib inline

import time
import os
import cv2
import random
import imageio

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

from tqdm import tqdm
from io import BytesIO

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

## BUG MEMORY, REFAIRE LA DATA AUGMENTATION SANS TUER LA MEMOIRE !!!

# Global variables

In [2]:
LABELS = ['FINGER', 'FIST', 'LEFT', 'PALM', 'RIGHT', 'EMPTY']
N_LABELS = len(LABELS)
INPUT_SIZE = 128 # width and height
OS = "linux"
TPU = True
VALID_SIZE = 0.2
VIDEO_PATH = "../input/hand-gesture/data_background/test.mkv" ## On kaggle
# VIDEO_PATH = "data_background/test.mkv" ## On computer
# label_name = ["palm_horizontal", "L", "fist_horizontal", "fist_vertical", "thumb_up", "index", "ok", "palm_vertical", "C", "thumb_down"]

In [3]:
if TPU:
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Device:', tpu.master())
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    except:
        strategy = tf.distribute.get_strategy()
    print('Number of replicas:', strategy.num_replicas_in_sync)

    AUTOTUNE = tf.data.experimental.AUTOTUNE
    BATCH_SIZE = 16 * strategy.num_replicas_in_sync
else:
    BATCH_SIZE = 128

# Loading data

In [4]:
def load_labelling_data():
    X = []
    y = []
    if OS == "windows":
        split_ = "\\"
    else:
        split_ = "/"
    for root, _, files in tqdm(os.walk("../", topdown=False)): 
        for name in files:
            path = os.path.join(root, name)
            if path.endswith("jpg"):
                # if path.split(split_)[-1][0] != ".":
                if path.split(split_)[-1][0].isalpha():
                    # Loading images
                    img = cv2.imread(path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    X.append(img)
                    # Loading labels
                    category = path.split(split_)[-1].split("_")[0]
                    # label = int(category.split("_")[0]) - 1
                    y.append(category)
    X = np.array(X)
    y = np.array(y)
    return X[:,:,:,np.newaxis], y

def load_test_data(width, height):
    X = []
    if OS == "windows":
        split_ = "\\"
    else:
        split_ = "/"
    for root, _, files in tqdm(os.walk("../", topdown=False)): 
        for name in files:
            path = os.path.join(root, name)
            if path.endswith("jpg"):
                # if path.split(split_)[-1][0] != ".":
                if not path.split(split_)[-1][0].isalpha() and path.split(split_)[-1][0] != ".":
                    # Loading images
                    img = cv2.imread(path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    img = cv2.resize(img, (width, height))
                    X.append(img)
    X = np.array(X)
    return X.reshape(X.shape[0], height, width, 1)

def get_background_images():
    images = []
    vidcap = cv2.VideoCapture(VIDEO_PATH)
    success,image = vidcap.read()
    while success:
        img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)[:,:,np.newaxis]
        images.append(img)
        success, image = vidcap.read()
    return np.array(images)

# def load_dataset_data(width, height):
#     X = []
#     y = []
#     if OS == "windows":
#         split_ = "\\"
#     else:
#         split_ = "/"
#     for root, dirs, files in tqdm(os.walk(".", topdown=False)): 
#         for name in files:
#             path = os.path.join(root, name)
#             if path.endswith("png"):
#                 # Loading labels
#                 category = path.split(split_)[4]
#                 label = int(category.split("_")[0]) - 1
#                 y.append(label)
#                 # Loading images
#                 img = cv2.imread(path)
#                 img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#                 img = cv2.resize(img, (width, height))
#                 X.append(img)
#     X = np.array(X)
#     y = np.array(y)
#     return X.reshape(X.shape[0], height, width, 1), y

In [5]:
X_labelling, y_labelling = load_labelling_data()
background_img = get_background_images()
background_images = background_img[::50]

In [6]:
print(X_labelling.shape)
print(y_labelling.shape)
print(background_images.shape)

# Data augmentation

In [7]:
plt.imshow(X_labelling[0])
plt.show()
plt.imshow(background_images[0])
plt.show()

In [8]:
def fill(img, h, w):
    img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
    return img
        
def horizontal_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = w*ratio
    if ratio > 0:
        img = img[:, :int(w-to_shift), :]
    if ratio < 0:
        img = img[:, int(-1*to_shift):, :]
    img = fill(img, h, w)
    return img

def vertical_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :, :]
    if ratio < 0:
        img = img[int(-1*to_shift):, :, :]
    img = fill(img, h, w)
    return img

def brightness(img, low, high):
    value = random.uniform(low, high)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255]  = 255
    hsv[:,:,2] = hsv[:,:,2]*value 
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

def zoom(img, value):
    if value > 1 or value < 0:
        return img
    value = random.uniform(value, 1)
    h, w = img.shape[:2]
    h_taken = int(value*h)
    w_taken = int(value*w)
    h_start = random.randint(0, h-h_taken)
    w_start = random.randint(0, w-w_taken)
    img = img[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
    img = fill(img, h, w)
    return img

def channel_shift(img, value):
    value = int(random.uniform(-value, value))
    img = img + value
    img[:,:,:][img[:,:,:]>255]  = 255
    img[:,:,:][img[:,:,:]<0]  = 0
    img = img.astype(np.uint8)
    return img.squeeze()

def horizontal_flip(img, flag):
    if flag:
        return cv2.flip(img, 1)
    else:
        return img

def vertical_flip(img, flag):
    if flag:
        return cv2.flip(img, 0)
    else:
        return img

def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img

def get_augmented_images(X, y):
    X_augmented = []
    y_augmented = []
    for i in range(X.shape[0]):
        X_augmented.append(X[i].squeeze())
        y_augmented.append(y[i])
        X_augmented.append(horizontal_shift(X[i], 0.7))
        y_augmented.append(y[i])
        X_augmented.append(vertical_shift(X[i], 0.7))
        y_augmented.append(y[i])
        # X_augmented.append(brightness(X[i], 0.5, 3))
        # y_augmented.append(y[i])
        X_augmented.append(zoom(X[i], 0.5))
        y_augmented.append(y[i])
        X_augmented.append(channel_shift(X[i], 60))
        y_augmented.append(y[i])
        # X_augmented.append(horizontal_flip(X[i], True))
        # y_augmented.append(y[i])
        # X_augmented.append(vertical_flip(X[i], True))
        # y_augmented.append(y[i])
        X_augmented.append(rotation(X[i], 20))
        y_augmented.append(y[i])
        X_augmented.append(rotation(X[i], 40))
        y_augmented.append(y[i])
        X_augmented.append(rotation(X[i], 60))
        y_augmented.append(y[i])
        X_augmented.append(rotation(X[i], -20))
        y_augmented.append(y[i])
        X_augmented.append(rotation(X[i], -40))
        y_augmented.append(y[i])
        X_augmented.append(rotation(X[i], -60))
        y_augmented.append(y[i])
    return np.array(X_augmented), np.array(y_augmented)

def get_augmented_background(X, n):
    X_augmented = []
    y_augmented = []
    cpt = 0
    for i in range(X.shape[0]):
        if cpt == n:
            break
        X_augmented.append(X[i].squeeze())
#         X_augmented.append(horizontal_shift(X[i], 0.7))
#         X_augmented.append(vertical_shift(X[i], 0.7))
        # X_augmented.append(brightness(X[i], 0.5, 3))
#         X_augmented.append(zoom(X[i], 0.5))
        X_augmented.append(channel_shift(X[i], 60))
        X_augmented.append(horizontal_flip(X[i], True))
        X_augmented.append(vertical_flip(X[i], True))
        X_augmented.append(rotation(X[i], 20))
        X_augmented.append(rotation(X[i], 40))
        X_augmented.append(rotation(X[i], 60))
        X_augmented.append(rotation(X[i], -20))
        X_augmented.append(rotation(X[i], -40))
        X_augmented.append(rotation(X[i], -60))
        cpt += 10
    new_X_augmented = []
    for i in range(X_augmented.shape[0]):
        new_X_augmented.append(cv2.resize(X_augmented[i], (INPUT_SIZE, INPUT_SIZE)))
    new_X_augmented = np.array(new_X_augmented)
    return np.array(new_X_augmented)

def add_background_to_data(background_images, hand_images, labels):
    final_images = []
    new_labels = []
    for idx, hand_img in enumerate(tqdm(hand_images)):
        for background_img in background_images:
            for r in range(1, 5):
                resize = int(hand_img.shape[0]*r)
                hand_img_resized = cv2.resize(hand_img, (resize, resize))[:,:,np.newaxis]
                y_offset = np.random.randint(background_img.shape[0] - hand_img_resized.shape[0])
                x_offset = np.random.randint(background_img.shape[1] - hand_img_resized.shape[1])
                tmp = background_img.copy()
                tmp[y_offset:y_offset+hand_img_resized.shape[0], x_offset:x_offset+hand_img_resized.shape[1]] = hand_img_resized
                tmp = cv2.resize(tmp, (INPUT_SIZE, INPUT_SIZE))[:,:,np.newaxis]
                final_images.append(tmp)
                new_labels.append(labels[idx])
    return np.array(final_images), np.array(new_labels)

In [9]:
X_labelling_augmented, y_labelling_augmented = get_augmented_images(X_labelling, y_labelling)
print(X_labelling_augmented.shape)
print(y_labelling_augmented.shape)


In [10]:
X, y = add_background_to_data(background_images, X_labelling_augmented, y_labelling_augmented)
print(X.shape)
print(y.shape)

In [ ]:
X_empty = get_augmented_background(background_images, int(X_labelling_augmented.shape[0]/N_LABELS))
y_empty = np.full((X.shape), N_LABELS)
print(X_empty.shape)
print(y_empty.shape)

In [ ]:
print(X.shape)
print(X_empty.shape)
np.concatenate((X, X_empty), axis=0)
np.concatenate((Y, Y_empty), axis=0)

In [ ]:
for _ in range(20):
    idx = np.random.randint(X.shape[0])
    plt.imshow(X[idx])
    plt.show()

In [ ]:
label_names, y = np.unique(y, return_inverse=True) ## Converts to categorical int and get label names
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=VALID_SIZE, random_state=42)

In [ ]:
X_train, X_valid, y_train, y_valid = X_train[::4], X_valid[::4], y_train[::4], y_valid[::4]

In [ ]:
X_train.shape

# LIMIT 16 GB MEMORY ~ (55000, 128, 128, 1)

# Building model

In [ ]:
def build_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(X_train[0].shape))) 
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
#     model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu')) 
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(N_LABELS, activation='softmax'))
    return model

def get_callbacks():
    early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                             min_delta=0,
                                             patience=15,
                                             verbose=2,
                                             mode='auto',
                                             baseline=None,
                                             restore_best_weights=True
                                             )
#     reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
#                                                   factor=0.1,
#                                                   patience=15,
#                                                   verbose=2,
#                                                   mode='auto',
#                                                   min_delta=0.0001,
#                                                   cooldown=0,
#                                                   min_lr=0
#                                                  )
#     return [early, reduce]
    return [early]

# Training model

In [ ]:
if TPU :
    with strategy.scope():       
        model = build_model()
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        model.summary()
else :
    model = build_model()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    epochs=200, 
                    batch_size=BATCH_SIZE, 
                    verbose=2, 
                    validation_data=(X_valid, y_valid), 
                    callbacks=get_callbacks())

In [ ]:
model.save('kaggle_model.h5')

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

# Inference

In [ ]:
X_test = load_test_data(INPUT_SIZE, INPUT_SIZE)

In [ ]:
plt.imshow(X_test[0])
plt.show()

In [ ]:
preds = np.argmax(model.predict(X_test), axis=1)
label_names

In [ ]:
for i in range(len(preds)):
    print("=======================================")
    plt.imshow(X_test[i])
    plt.show()
    print(label_names[preds[i]])

In [ ]:
inference_ = False

if inference_ :
    cv2.destroyAllWindows()
    before = time.time()
    vid = cv2.VideoCapture(1)
    while True:

        ret, frame = vid.read()
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (INPUT_SIZE, INPUT_SIZE))
        img = img.reshape(1, INPUT_SIZE, INPUT_SIZE, 1)

        after = int(time.time() - before)
        print(label_names[np.argmax(model.predict(img))], end='\r')

    vid.release()
    cv2.destroyAllWindows()